In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tensorflow import keras
import tensorflow as tf
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Activation, BatchNormalization, Flatten, GlobalAveragePooling2D,Dropout
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.applications.xception import Xception, preprocess_input

In [ ]:
!pip install py7zr

In [ ]:
from py7zr import unpack_7zarchive
import shutil

shutil.register_unpack_format('7zip',['.7z'],unpack_7zarchive) #Run for first time only

In [ ]:
shutil.unpack_archive('../input/cifar-10/train.7z', '/kaggle/temp/')

In [ ]:
train_labels = pd.read_csv("../input/cifar-10/trainLabels.csv", header="infer")

classes = train_labels['label'].unique()
print(classes)

In [ ]:
if not os.path.exists("/kaggle/temp/valid"):
    os.mkdir("/kaggle/temp/valid")
    
parent_path_train = "/kaggle/temp/train"
parent_path_valid = "/kaggle/temp/valid"
parent_path_test = "/kaggle/temp/test"

for class1 in classes:
    path_train = os.path.join(parent_path_train,class1)
    if not os.path.exists(path_train):
        os.mkdir(path_train)
    path_valid = os.path.join(parent_path_valid,class1)
    if not os.path.exists(path_valid):
        os.mkdir(path_valid)
        
for (int_ind,row) in train_labels.iterrows():
    id = str(row["id"])+".png"
    source_path = os.path.join(parent_path_train,id)
    
    p=np.random.random()
    if p<=0.8:
        target_path = os.path.join(parent_path_train,row["label"],id)
        os.replace(source_path, target_path)
    else:
        target_path = os.path.join(parent_path_valid,row["label"],id)
        os.replace(source_path, target_path)

In [ ]:
!ls /kaggle/temp/valid
!ls /kaggle/temp/train

In [ ]:
xception_wo_top=Xception(include_top=False, input_shape=(291,291,3), weights='imagenet')
xception_wo_top.summary()

for layer in xception_wo_top.layers:
    layer.trainable=True

In [ ]:
classifier = Sequential()
classifier.add(xception_wo_top)
classifier.add(GlobalAveragePooling2D())
classifier.add(Dropout(0.3))
classifier.add(Dense(10,activation='softmax'))

classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.summary()

for i in classifier.layers:
    print(i.trainable)

In [ ]:
reduceLROnPlateau=ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

In [ ]:
train_datagen = ImageDataGenerator(featurewise_center=False,
                             samplewise_center=False,
                             featurewise_std_normalization=False,
                             samplewise_std_normalization=False,
                             zca_whitening=False,
                             rotation_range=10,
                             zoom_range=0.1,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             horizontal_flip=False,
                             vertical_flip=False,
                             preprocessing_function=preprocess_input,
                             rescale=None)

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = train_datagen.flow_from_directory(directory='/kaggle/temp/train/', 
                                                    shuffle=True, target_size=(192,192),
                                                    batch_size=128)
valid_generator = valid_datagen.flow_from_directory(directory='/kaggle/temp/valid/',
                                                    shuffle=True, target_size=(192,192),
                                                    batch_size=128)

In [ ]:
classifier.fit(train_generator,
               epochs=8, 
               validation_data=valid_generator,
               callbacks=[reduceLROnPlateau],
               steps_per_epoch=train_generator.n//train_generator.batch_size,
               validation_steps= valid_generator.n//valid_generator.batch_size,
               workers=8,
               use_multiprocessing=True)

In [ ]:
shutil.unpack_archive('/kaggle/input/cifar-10/test.7z','/kaggle/temp/test')

In [ ]:
shutil.unregister_unpack_format('7zip')

In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_gen = test_datagen.flow_from_directory(directory='/kaggle/temp/test',target_size=(192,192),batch_size=64,class_mode=None,shuffle=False)

In [ ]:
test_gen.reset()
predictions_vecs = classifier.predict(test_gen)

predictions_final = np.argmax(predictions_vecs, axis=1)

In [ ]:
print(type(train_generator.class_indices))
print(train_generator.class_indices)

classes = {value:key for (key,value) in train_generator.class_indices.items()}
print(classes)

predicted_classes=np.empty(shape=300000,dtype=np.dtype('U20'))

ind=0
for i in predictions_final.tolist():
    predicted_classes[ind]=classes[i]
    ind=ind+1
    
filenames_wo_ext = []
for fname in test_gen.filenames:
    filenames_wo_ext.append(int(fname.split(sep="/")[1].split(sep=".")[0])-1)

predicted_classes_final = np.empty(shape=300000,dtype=np.dtype('U20'))
predicted_classes_final[filenames_wo_ext]=predicted_classes

In [ ]:
sub = pd.read_csv('../input/cifar-10/sampleSubmission.csv',header='infer')
sub.info()

In [ ]:
sub['label'] = predicted_classes_final
sub.to_csv('19BCE226_submission.csv',index=False)